<a href="https://colab.research.google.com/github/Vitorhugofsousa/pharma_NE_MM/blob/main/df_gen/gerar_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Criando uma tabela ficticia
O celula de codigo abaixo tem como objetivo criar um dataframe ficticio sobre a distribuição de medicamentos entre redes de farmacias que compõem a empresa ficticia **Pharma Nordeste**, declarando as cidades onde a empresa atua, as redes de farmacias que fazem parte desse conglomerado, os laboratorios ficticios que vendem medicamentos para as farmacias que fazem parte do grupo Pharma Nordeste assim como o nome dos medicamentos e suas respectivas dosagens.

o objetivo desse dataframe é gerar uma base de dados para realização dos processos de engenharia de dados e analise de dados, para resolver os problemas e dores que da referenciada empresa ficticia.


Essa tabela será usada como uma tabela fato dentro de um modelo.

In [ ]:
import pandas as pd
import numpy as np
import random
import string

# --- CONFIGURAÇÕES GERAIS ---
NUM_ROWS = 15000
random.seed(42)  # Reprodutibilidade

# --- 1. DADOS DE REFERÊNCIA ---

cidades_ne = [
    "Recife", "Salvador", "Fortaleza", "São Luís", "Maceió", "João Pessoa",
    "Teresina", "Natal", "Aracaju", "Olinda", "Caruaru", "Campina Grande",
    "Feira de Santana", "Petrolina", "Juazeiro do Norte", "Sobral", "Arapiraca"
]

#redes ficticias de farmacias que pertencem a uma unica empresa ficticia
nomes_farmacias_base = [
    "Farmácia Pague Menos", "Drogasil", "Farmácia Permanente", "Drogaria São Paulo",
    "Big Ben", "Farmácias Independente", "Extra Farma", "Drogarias Globo"
]

laboratorios_ficticios = [
    "NeoQuímica", "Medley", "EMS", "Eurofarma",
    "Teuto", "Prati-Donaduzzi", "Aché", "Sandoz"
]

# DICIONÁRIO MESTRE DE COERÊNCIA
#regras reais: Nome -> Substância -> Formas Válidas -> Dosagens Válidas
medicamentos_templates = [
    {
        "nome": "Dipirona Monohidratada",
        "substancia": "Analgésico e Antitérmico",
        "formas": [
            {"tipo": "Comprimido", "dosagens": ["500mg", "1g"]},
            {"tipo": "Solução Oral (Gotas)", "dosagens": ["500mg/ml"]},
            {"tipo": "Solução Injetável", "dosagens": ["500mg/ml"]}
        ]
    },
    {
        "nome": "Amoxicilina",
        "substancia": "Antibiótico",
        "formas": [
            {"tipo": "Cápsula", "dosagens": ["500mg", "875mg"]},
            {"tipo": "Suspensão Oral", "dosagens": ["250mg/5ml", "500mg/5ml"]}
        ]
    },
    {
        "nome": "Losartana Potássica",
        "substancia": "Anti-hipertensivo",
        "formas": [
            {"tipo": "Comprimido", "dosagens": ["50mg", "100mg"]}
        ]
    },
    {
        "nome": "Diclofenaco Dietilamônio",
        "substancia": "Anti-inflamatório Tópico",
        "formas": [
            {"tipo": "Gel Creme", "dosagens": ["11.6mg/g"]} # Padrão de pomada
        ]
    },
    {
        "nome": "Ibuprofeno",
        "substancia": "Anti-inflamatório",
        "formas": [
            {"tipo": "Comprimido Revestido", "dosagens": ["400mg", "600mg"]},
            {"tipo": "Suspensão Oral (Gotas)", "dosagens": ["50mg/ml", "100mg/ml"]}
        ]
    },
    {
        "nome": "Omeprazol",
        "substancia": "Antiulceroso",
        "formas": [
            {"tipo": "Cápsula", "dosagens": ["20mg", "40mg"]}
        ]
    },
    {
        "nome": "Clonazepam",
        "substancia": "Ansiolítico",
        "formas": [
            {"tipo": "Comprimido", "dosagens": ["0.5mg", "2.0mg"]},
            {"tipo": "Solução Oral (Gotas)", "dosagens": ["2.5mg/ml"]}
        ]
    },
    {
        "nome": "Cetoconazol",
        "substancia": "Antifúngico",
        "formas": [
            {"tipo": "Creme Dermatológico", "dosagens": ["20mg/g"]},
            {"tipo": "Shampoo", "dosagens": ["20mg/g"]},
            {"tipo": "Comprimido", "dosagens": ["200mg"]}
        ]
    },
    {
        "nome": "Insulina Humana NPH",
        "substancia": "Hormônio Antidiabético",
        "formas": [
            {"tipo": "Frasco-Ampola", "dosagens": ["100UI/ml"]},
            {"tipo": "Refil Caneta", "dosagens": ["100UI/ml"]}
        ]
    },
    {
        "nome": "Simeticona",
        "substancia": "Antigases",
        "formas": [
            {"tipo": "Cápsula Gelatinosa", "dosagens": ["125mg"]},
            {"tipo": "Emulsão Oral (Gotas)", "dosagens": ["75mg/ml"]}
        ]
    }
]

# --- 2. GERAÇÃO DE CATÁLOGOS ÚNICOS ---

# Catálogo de Farmácias (1 ID único por loja física)
farmacias_db = []
for i in range(1, 601): # 600 Farmácias
    nome_base = random.choice(nomes_farmacias_base)
    bairro_suffix = random.choice(string.ascii_uppercase)
    farmacias_db.append({
        "id_farmacia": 20000 + i,
        "nome_farmacia": f"{nome_base} - Unidade {bairro_suffix}{i}",
        "cidade": random.choice(cidades_ne)
    })

# Catálogo de Produtos (SKUs)
# Expandimos o dicionário mestre criando versões de diferentes fabricantes
produtos_db = []
id_med_counter = 700000

for item in medicamentos_templates:
    for forma in item['formas']:
        for dosagem in forma['dosagens']:
            # Para cada combinação real, sorteamos 3 a 5 fabricantes que "produzem" isso
            fabricantes_deste_item = random.sample(laboratorios_ficticios, k=random.randint(3, 5))

            for fab in fabricantes_deste_item:
                produtos_db.append({
                    "id_medicamento_unico": id_med_counter,
                    "nome_medicamento": item['nome'],
                    "tipo_substancia": item['substancia'],
                    "apresentacao": forma['tipo'],
                    "dosagem": dosagem,
                    "fabricante": fab
                })
                id_med_counter += 1

# --- 3. GERAÇÃO DO DATASET DE ESTOQUE (LINHAS) ---

data = []

for _ in range(NUM_ROWS):
    # Escolhe uma farmácia aleatória
    farmacia = random.choice(farmacias_db)

    # Escolhe um produto aleatório (mas coerente) do catálogo
    produto = random.choice(produtos_db)

    # Lógica de Quantidade baseada no tipo (Estoque)
    if "Antibiótico" in produto['tipo_substancia'] or "Injetável" in produto['apresentacao']:
        qtd = random.randint(5, 40) # Estoque menor para controlados/injetáveis
    elif "Comprimido" in produto['apresentacao']:
        qtd = random.randint(20, 300) # Estoque alto para caixas de comprimidos
    else:
        qtd = random.randint(10, 100) # Estoque médio para cremes/xaropes

    row = {
        "cod_medicamento": produto["id_medicamento_unico"],
        "nome_medicamento": produto["nome_medicamento"],
        "substancia_principal": produto["tipo_substancia"],
        "apresentacao": produto["apresentacao"],
        "dosagem": produto["dosagem"],
        "fabricante": produto["fabricante"],
        "quantidade_estoque": qtd,
        "cod_farmacia": farmacia["id_farmacia"],
        "nome_farmacia": farmacia["nome_farmacia"],
        "cidade": farmacia["cidade"]
    }
    data.append(row)

df = pd.DataFrame(data)

# --- 4. INTRODUÇÃO DE IMPERFEIÇÕES (QUALIDADE DE DADOS) ---

# Função para adicionar ruído
def introduzir_erro(valor, chance=0.03, tipo='texto'):
    if random.random() < chance:
        if tipo == 'nan': return np.nan
        if tipo == 'texto_upper': return str(valor).upper()
        if tipo == 'texto_lower': return str(valor).lower()
    return valor

# A. Valores Nulos (Missing Data) em campos não-chave
df['fabricante'] = df['fabricante'].apply(lambda x: introduzir_erro(x, chance=0.05, tipo='nan'))
df['apresentacao'] = df['apresentacao'].apply(lambda x: introduzir_erro(x, chance=0.03, tipo='nan'))

# B. Inconsistência de Padronização (Ex: Recife vs RECIFE vs recife)
df['cidade'] = df['cidade'].apply(lambda x: introduzir_erro(x, chance=0.10, tipo='texto_upper'))
df['cidade'] = df['cidade'].apply(lambda x: introduzir_erro(x, chance=0.05, tipo='texto_lower'))

# C. Erros de Input na Quantidade (Tipo Errado ou Valor Ilógico)
def sujar_quantidade(val):
    prob = random.random()
    if prob < 0.005: return -val # Valor negativo (erro lógico)
    if prob < 0.005: return 999999 # Outlier extremo
    return val

df['quantidade_estoque'] = df['quantidade_estoque'].apply(sujar_quantidade)

# --- 5. EXPORTAÇÃO ---
print(f"Dataset gerado com {len(df)} registros.")
print(df.head(10))

filename = "distribuicao_medicamentos_nordeste_v2.csv"
df.to_csv(filename, index=False, encoding='utf-8')
print(f"Arquivo '{filename}' pronto para download.")

Dataset gerado com 15000 registros.
   cod_medicamento          nome_medicamento      substancia_principal  \
0           700049                Ibuprofeno         Anti-inflamatório   
1           700020               Amoxicilina               Antibiótico   
2           700092               Cetoconazol               Antifúngico   
3           700097       Insulina Humana NPH    Hormônio Antidiabético   
4           700065                 Omeprazol              Antiulceroso   
5           700039  Diclofenaco Dietilamônio  Anti-inflamatório Tópico   
6           700011    Dipirona Monohidratada  Analgésico e Antitérmico   
7           700058                 Omeprazol              Antiulceroso   
8           700074                Clonazepam               Ansiolítico   
9           700006    Dipirona Monohidratada  Analgésico e Antitérmico   

           apresentacao   dosagem                  fabricante  \
0  Comprimido Revestido     600mg             Aché (Fictício)   
1               Cáp

##Criando uma tabela fato adicional responsavel pelo historico de remessas

Essa tabela tem como objetivo fornecer contexto adicional para a tabela principal, ao permitir indicar quantas farmacias venderam em relação ao seu estoque inicial, ou possíveis desequilibrios na alocação adequada.

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

distrib = pd.read_csv("/content/distribuicao_medicamentos_nordeste_v2.csv")

df_remessas = distrib[['cod_farmacia', 'cod_medicamento', 'quantidade_estoque']].copy()

df_remessas.rename(columns={'quantidade_estoque': 'estoque_atual_bruto'}, inplace=True)

# --- 2. REGRA DE NEGÓCIO: ESTOQUE INICIAL > ATUAL ---

# Se for NaN ou texto sujo, assumimos 0 para conseguir somar. Se for negativo, assumimos 0.
def limpar_para_calculo(val):
    try:
        val = float(val)
        return val if val > 0 else 0
    except:
        return 0

# Cria uma série temporária limpa
estoque_limpo = df_remessas['estoque_atual_bruto'].apply(limpar_para_calculo)

# Passo B: Gerar a "Quantidade Vendida" aleatória (entre 1 e 50 unidades vendidas desde a remessa)
vendas_simuladas = np.random.randint(1, 50, size=len(df_remessas))

# Passo C: Calcular a Quantidade da Remessa Original
# Remessa = O que tem hoje + O que foi vendido. Logo, Remessa > Hoje.
df_remessas['quantidade_recebida'] = estoque_limpo + vendas_simuladas

# Converter para int (removendo decimais do cálculo)
df_remessas['quantidade_recebida'] = df_remessas['quantidade_recebida'].astype(int)


# --- 3. DADOS ADICIONAIS DA REMESSA ---

# Gerar Datas (Últimos 45 dias)
def gerar_data():
    dias_atras = random.randint(1, 45)
    data = datetime.now() - timedelta(days=dias_atras)
    return data.strftime("%Y-%m-%d")

df_remessas['data_entrega'] = [gerar_data() for _ in range(len(df_remessas))]

# Gerar Número do Lote (Alfanumérico)
def gerar_lote():
    letras = "".join(random.choices("ABCDEF", k=2))
    nums = "".join(random.choices("0123456789", k=4))
    return f"{letras}{nums}"

df_remessas['lote_fabricacao'] = [gerar_lote() for _ in range(len(df_remessas))]

# Gerar Custo Unitário (R$)
# Aleatório entre 2.50 e 150.00 para simular custos variados
df_remessas['custo_unitario'] = np.round(np.random.uniform(2.50, 150.00, size=len(df_remessas)), 2)


# --- 4. INTRODUZIR IMPERFEIÇÕES (SUJEIRA) ---

# A. Datas vazias ou futuras (erros de registro)
def sujar_data(val):
    prob = random.random()
    if prob < 0.02: return "" # Data vazia
    if prob < 0.02: return "2030-01-01" # Data futura impossível
    return val

df_remessas['data_entrega'] = df_remessas['data_entrega'].apply(sujar_data)

# B. Custos Zerados ou Negativos
def sujar_custo(val):
    prob = random.random()
    if prob < 0.03: return 0.00 # Custo zero
    if prob < 0.01: return -val # Custo negativo
    return val

df_remessas['custo_unitario'] = df_remessas['custo_unitario'].apply(sujar_custo)


# --- 5. LIMPEZA FINAL E EXPORTAÇÃO ---

# Removemos a coluna 'estoque_atual_bruto' pois ela pertence à outra tabela,
# trouxemos apenas para fazer o cálculo matemático.
df_remessas = df_remessas.drop(columns=['estoque_atual_bruto'])

# Adicionamos um ID único para essa transação de remessa
df_remessas['id_transacao_remessa'] = range(1000000, 1000000 + len(df_remessas))

# Reordenar colunas
colunas_ordem = [
    'id_transacao_remessa', 'cod_farmacia', 'cod_medicamento',
    'quantidade_recebida', 'data_entrega', 'lote_fabricacao', 'custo_unitario'
]
df_remessas = df_remessas[colunas_ordem]

print(f"Tabela de Remessas gerada com {len(df_remessas)} linhas.")
print(df_remessas.head())

# Salvar CSV
filename_remessa = "historico_remessas_nordeste.csv"
df_remessas.to_csv(filename_remessa, index=False, encoding='utf-8')
print(f"Arquivo '{filename_remessa}' pronto para download.")

Tabela de Remessas gerada com 15000 linhas.
   id_transacao_remessa  cod_farmacia  cod_medicamento  quantidade_recebida  \
0               1000000         20298           700049                  272   
1               1000001         20255           700020                   38   
2               1000002         20592           700092                  211   
3               1000003         20195           700097                   33   
4               1000004         20511           700065                   78   

  data_entrega lote_fabricacao  custo_unitario  
0   2025-12-15          DA3796           88.92  
1   2026-01-02          DC0516           47.75  
2   2025-12-17          DA1633           39.55  
3   2025-12-27          FF7354           57.34  
4   2025-12-18          BD1623           10.69  
Arquivo 'historico_remessas_nordeste.csv' pronto para download.


#A Célula de codigo abaixo representa a criação da Tabela dimensão para fornecer contexto de localização.

In [4]:
import pandas as pd
import random

# --- 1. CONFIGURAÇÃO DA DISTRIBUIÇÃO (Regras de Negócio) ---

# A. CAPITAIS (Total: 90)
# Distribuição desproporcional baseada no tamanho aproximado da cidade/mercado
distribuicao_capitais = {
    "Recife": 14,
    "Salvador": 14,
    "Fortaleza": 14,
    "São Luís": 9,
    "Maceió": 9,
    "Natal": 8,
    "João Pessoa": 8,
    "Teresina": 7,
    "Aracaju": 7
}
# Verificação de segurança: 14*3 + 9*2 + 8*2 + 7*2 = 42 + 18 + 16 + 14 = 90. Correto.

# B. INTERIOR (Total: 18 em 8 Cidades)
# Regra: Min 2, Max 3.
# Lógica: 8 cidades * 2 = 16 farmácias. Sobram 2 vagas.
# Sorteamos 2 cidades para terem 3 farmácias, as outras ficam com 2.
cidades_interior = [
    "Caruaru", "Campina Grande", "Petrolina", "Feira de Santana",
    "Mossoró", "Juazeiro do Norte", "Sobral", "Arapiraca"
]
random.shuffle(cidades_interior)
distribuicao_interior = {cidade: 2 for cidade in cidades_interior}
# As duas primeiras da lista sorteada ganham +1
distribuicao_interior[cidades_interior[0]] += 1
distribuicao_interior[cidades_interior[1]] += 1


# --- 2. BANCO DE DADOS DE ENDEREÇOS REAIS E VÁLIDOS ---
# (Formato: Bairro, Logradouro, CEP Real)

db_enderecos_reais = {
    "Recife": [
        ("Boa Viagem", "Avenida Boa Viagem", "51020-000"),
        ("Boa Viagem", "Rua dos Navegantes", "51020-010"),
        ("Boa Viagem", "Rua Barão de Souza Leão", "51030-300"),
        ("Boa Viagem", "Rua Setúbal", "51030-010"),
        ("Pina", "Avenida Herculano Bandeira", "51110-130"),
        ("Madalena", "Rua Real da Torre", "50610-000"),
        ("Madalena", "Rua Benfica", "50720-001"),
        ("Casa Forte", "Avenida 17 de Agosto", "52060-590"),
        ("Casa Forte", "Praça de Casa Forte", "52061-420"),
        ("Espinheiro", "Rua da Hora", "52020-010"),
        ("Derby", "Praça do Derby", "52010-140"),
        ("Centro", "Avenida Conde da Boa Vista", "50060-002"),
        ("Centro", "Rua da Aurora", "50050-000"),
        ("Várzea", "Avenida Caxangá", "50740-000")
    ],
    "Salvador": [
        ("Barra", "Avenida Oceânica", "40140-130"),
        ("Barra", "Rua Marquês de Caravelas", "40140-240"),
        ("Rio Vermelho", "Rua da Paciência", "41950-010"),
        ("Rio Vermelho", "Rua Oswaldo Cruz", "41940-000"),
        ("Pituba", "Avenida Paulo VI", "41810-000"),
        ("Pituba", "Avenida Manoel Dias da Silva", "41830-000"),
        ("Pituba", "Rua Ceará", "41830-450"),
        ("Itaigara", "Avenida Antônio Carlos Magalhães", "41825-000"),
        ("Brotas", "Avenida Dom João VI", "40285-000"),
        ("Centro", "Avenida Joana Angélica", "40050-001"),
        ("Centro", "Rua Carlos Gomes", "40060-330"),
        ("Itapuã", "Avenida Dorival Caymmi", "41635-150"),
        ("Ondina", "Avenida Adhemar de Barros", "40170-110"),
        ("Caminho das Árvores", "Rua da Alfazema", "41820-710")
    ],
    "Fortaleza": [
        ("Meireles", "Avenida Beira Mar", "60165-121"),
        ("Meireles", "Rua Silva Jatahy", "60165-070"),
        ("Meireles", "Avenida da Abolição", "60165-081"),
        ("Aldeota", "Avenida Santos Dumont", "60150-160"),
        ("Aldeota", "Avenida Dom Luís", "60355-610"),
        ("Aldeota", "Rua Leonardo Mota", "60170-040"),
        ("Centro", "Rua Barão do Rio Branco", "60025-060"),
        ("Centro", "Rua Major Facundo", "60025-100"),
        ("Benfica", "Avenida da Universidade", "60020-181"),
        ("Papicu", "Avenida Engenheiro Santana Júnior", "60175-285"),
        ("Fátima", "Avenida 13 de Maio", "60040-530"),
        ("Praia de Iracema", "Av. Historiador Raimundo Girão", "60165-050"),
        ("Parquelândia", "Avenida Jovita Feitosa", "60455-410"),
        ("Dionísio Torres", "Avenida Pontes Vieira", "60130-240")
    ],
    "São Luís": [
        ("Renascença", "Avenida Colares Moreira", "65075-441"),
        ("Renascença", "Rua das Mitras", "65075-770"),
        ("Calhau", "Avenida dos Holandeses", "65071-380"),
        ("Centro", "Rua Grande", "65020-250"),
        ("Centro", "Rua do Sol", "65020-590"),
        ("São Francisco", "Avenida Castelo Branco", "65076-090"),
        ("Cohama", "Avenida Daniel de La Touche", "65074-115"),
        ("Ponta d'Areia", "Avenida dos Holandeses", "65077-357"),
        ("Monte Castelo", "Avenida Getúlio Vargas", "65030-005")
    ],
    "Maceió": [
        ("Ponta Verde", "Avenida Sílvio Carlos Viana", "57035-160"),
        ("Ponta Verde", "Rua Durval Guimarães", "57035-060"),
        ("Ponta Verde", "Rua Engenheiro Mario de Gusmão", "57035-000"),
        ("Jatiúca", "Av. Dr. Antônio Gomes de Barros", "57036-000"),
        ("Pajuçara", "Av. Dr. Antônio Gouveia", "57030-170"),
        ("Farol", "Avenida Fernandes Lima", "57050-000"),
        ("Centro", "Rua do Comércio", "57020-000"),
        ("Mangabeiras", "Avenida Gustavo Paiva", "57031-530"),
        ("Gruta de Lourdes", "Avenida Rotary", "57052-480")
    ],
    "Natal": [
        ("Ponta Negra", "Avenida Engenheiro Roberto Freire", "59090-000"),
        ("Ponta Negra", "Rua Praia de Genipabu", "59094-010"),
        ("Tirol", "Avenida Hermes da Fonseca", "59020-000"),
        ("Tirol", "Avenida Afonso Pena", "59020-100"),
        ("Petrópolis", "Avenida Deodoro da Fonseca", "59020-600"),
        ("Lagoa Nova", "Avenida Prudente de Morais", "59063-200"),
        ("Alecrim", "Avenida Coronel Estevam", "59035-000"),
        ("Candelária", "Avenida da Integração", "59064-500")
    ],
    "João Pessoa": [
        ("Tambaú", "Avenida Almirante Tamandaré", "58039-010"),
        ("Tambaú", "Avenida Epitácio Pessoa", "58039-000"),
        ("Manaíra", "Avenida General Edson Ramalho", "58038-100"),
        ("Manaíra", "Avenida João Maurício", "58038-000"),
        ("Cabo Branco", "Avenida Cabo Branco", "58045-010"),
        ("Centro", "Parque Solon de Lucena", "58013-130"),
        ("Torre", "Avenida Beira Rio", "58040-480"),
        ("Bessa", "Avenida Presidente Afonso Pena", "58035-030")
    ],
    "Teresina": [
        ("Jockey", "Avenida Nossa Senhora de Fátima", "64048-100"),
        ("Jockey", "Avenida Dom Severino", "64048-200"),
        ("Centro", "Avenida Frei Serafim", "64001-020"),
        ("Centro", "Rua Álvaro Mendes", "64000-060"),
        ("Fátima", "Avenida Ininga", "64048-110"),
        ("São Cristóvão", "Avenida Presidente Kennedy", "64052-000"),
        ("Dirceu", "Rua Joaquim Nelson", "64077-500")
    ],
    "Aracaju": [
        ("Atalaia", "Avenida Santos Dumont", "49035-500"),
        ("Atalaia", "Rua Niceu Dantas", "49037-470"),
        ("Jardins", "Av. Min. Geraldo Barreto Sobral", "49026-010"),
        ("Grageru", "Avenida Hermes Fontes", "49025-000"),
        ("Centro", "Rua Itabaianinha", "49010-190"),
        ("13 de Julho", "Avenida Beira Mar", "49020-010"),
        ("Siqueira Campos", "Rua Santa Catarina", "49075-000")
    ],
    # INTERIOR
    "Caruaru": [
        ("Maurício de Nassau", "Avenida Agamenon Magalhães", "55012-290"),
        ("Centro", "Rua 15 de Novembro", "55002-010"),
        ("Universitário", "Avenida Portugal", "55016-400")
    ],
    "Campina Grande": [
        ("Centro", "Rua Maciel Pinheiro", "58400-100"),
        ("Prata", "Rua Rodrigues Alves", "58400-550"),
        ("Catolé", "Avenida Vigário Calixto", "58410-340")
    ],
    "Petrolina": [
        ("Centro", "Avenida Guararapes", "56302-000"),
        ("Areia Branca", "Avenida São Gonçalo", "56328-320"),
        ("Vila Eduardo", "Avenida da Integração", "56328-010")
    ],
    "Feira de Santana": [
        ("Centro", "Avenida Getúlio Vargas", "44001-525"),
        ("Brasília", "Avenida Maria Quitéria", "44001-008"),
        ("Capuchinhos", "Rua Presidente Dutra", "44052-015")
    ],
    "Mossoró": [
        ("Centro", "Avenida Augusto Severo", "59600-150"),
        ("Nova Betânia", "Avenida João da Escóssia", "59607-330"),
        ("Abolição", "Avenida Abel Coelho", "59611-100")
    ],
    "Juazeiro do Norte": [
        ("Centro", "Rua São Pedro", "63010-010"),
        ("Lagoa Seca", "Avenida Leão Sampaio", "63040-000"),
        ("Triângulo", "Avenida Padre Cícero", "63041-140")
    ],
    "Sobral": [
        ("Centro", "Rua Coronel José Sabóia", "62010-230"),
        ("Dom Expedito", "Avenida Monsenhor Aloísio Pinto", "62050-255"),
        ("Campo dos Velhos", "Avenida Dr. Guarany", "62010-300")
    ],
    "Arapiraca": [
        ("Centro", "Rua Estudante José de Oliveira Leite", "57300-030"),
        ("Alto do Cruzeiro", "Rua Delmiro Gouveia", "57312-230"),
        ("Brasília", "Rua Brasília", "57313-050")
    ]
}

# --- 3. CONSTRUÇÃO DA TABELA ---

farmacias_data = []
id_counter = 1001
nomes_rede = [
    "Farmácia Pague Mais", "DroBrasil", "Farmácia Não Permanente", "Drogaria São Pedro",
    "Drogarias Big Bom", "Farmácias Dependente", "Xtra Pharma", "Drogarias Globinho"
]

# Função auxiliar para criar linhas
def criar_farmacias(cidade, quantidade):
    global id_counter
    # Pega endereços reais disponíveis
    enderecos_disponiveis = db_enderecos_reais[cidade]

    # Validação de segurança: se pedirmos mais do que existe no DB, limita ao max do DB
    if quantidade > len(enderecos_disponiveis):
        quantidade = len(enderecos_disponiveis)

    # Seleciona endereços aleatórios sem reposição (para garantir unicidade)
    selecionados = random.sample(enderecos_disponiveis, quantidade)

    for bairro, logradouro, cep in selecionados:
        # Gera nome único
        nome_fantasia = f"{random.choice(nomes_rede)} - Unid. {id_counter}"
        # Gera número de porta aleatório
        numero = random.randint(10, 3500)

        farmacias_data.append({
            "cod_farmacia": id_counter,
            "nome_farmacia": nome_fantasia,
            "cidade": cidade,
            "bairro": bairro,
            "logradouro": logradouro,
            "numero": numero,
            "cep": cep
        })
        id_counter += 1

# Processar Capitais
print("--- Distribuindo Capitais ---")
for cidade, qtd in distribuicao_capitais.items():
    criar_farmacias(cidade, qtd)
    print(f"{cidade}: {qtd} farmácias geradas.")

# Processar Interior
print("\n--- Distribuindo Interior ---")
for cidade, qtd in distribuicao_interior.items():
    criar_farmacias(cidade, qtd)
    print(f"{cidade}: {qtd} farmácias geradas.")

# --- 4. DATAFRAME E EXPORTAÇÃO ---

df_final = pd.DataFrame(farmacias_data)

print(f"\nTotal de Farmácias Geradas: {len(df_final)}")
print(df_final.head())

# Verificação de Unicidade
duplicados = df_final[df_final.duplicated(subset=['logradouro', 'cep'], keep=False)]
if len(duplicados) == 0:
    print("\nSUCESSO: Todos os endereços são únicos e reais!")
else:
    print(f"\nERRO: Encontradas duplicatas.")

filename = "dim_farmacias_final_108.csv"
df_final.to_csv(filename, index=False, encoding='utf-8')
print(f"Arquivo salvo: {filename}")

--- Distribuindo Capitais ---
Recife: 14 farmácias geradas.
Salvador: 14 farmácias geradas.
Fortaleza: 14 farmácias geradas.
São Luís: 9 farmácias geradas.
Maceió: 9 farmácias geradas.
Natal: 8 farmácias geradas.
João Pessoa: 8 farmácias geradas.
Teresina: 7 farmácias geradas.
Aracaju: 7 farmácias geradas.

--- Distribuindo Interior ---
Arapiraca: 3 farmácias geradas.
Mossoró: 3 farmácias geradas.
Juazeiro do Norte: 2 farmácias geradas.
Sobral: 2 farmácias geradas.
Feira de Santana: 2 farmácias geradas.
Campina Grande: 2 farmácias geradas.
Petrolina: 2 farmácias geradas.
Caruaru: 2 farmácias geradas.

Total de Farmácias Geradas: 108
   cod_farmacia                         nome_farmacia  cidade      bairro  \
0          1001  Farmácia Não Permanente - Unid. 1001  Recife  Boa Viagem   
1          1002        Drogarias Big Bom - Unid. 1002  Recife  Casa Forte   
2          1003              Xtra Pharma - Unid. 1003  Recife      Centro   
3          1004       Drogarias Globinho - Unid. 10